In [ ]:

#Used for storing and playing with data using DataFrame.
import pandas as pd
#Used for working with arrays in python.
import numpy as np
#Used for plotting graphs from data.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
#Distrubation test
import scipy
from scipy.stats import skew, kurtosis, shapiro, jarque_bera

# Read csv data
data = pd.read_csv('RIA02.20230502T160508.csv')




### Number of Bedrooms
One bed :1
Two bed : 2
Three bed : 3
Four plus bed: 4 

### Property Type
Other flats: 1
Apartment: 2
Terrace house : 3
Semi detached house: 4
Detached house : 5

In [ ]:
#DAta Overview
data.head(10)

In [ ]:
data.columns

In [ ]:
# Detect null observation number and data type
data.info()

In [ ]:
#Overview for Numeric Variables 
# The data between 2008 to 2021 
data.describe().T

## Data Cleaning 

In [ ]:
# Drop rows which doesnt have rent price information
data.dropna(subset=["VALUE"],inplace=True)

In [ ]:
#Drop first and fifth row
data=data.iloc[:,[1,2,3,4,6]]

In [ ]:
#To checj the code
data.head()

In [ ]:
#Cleaning unneccesary rows because We analize the houses by feature 
data=data.loc[ (data["Number of Bedrooms"].isin(["One bed","Two bed","Three bed","Four plus bed"]))
              & (data["Property Type"] != "All property types")]



In [ ]:
#To get information just about Dublin
dub_data=data[data["Location"].str.contains("Dublin")]

In [ ]:
locations = sorted(dub_data['Location'].unique())
locations

In [ ]:
# this analays based on Dublin areas ( Dublin 22 has any unique information so Dublin 22 Clondalkin is used for Dublin 22 area. 
#Dublin 22 consist of Clondalkin, Liffey Valley, Newcastle and Neilstown)

In [ ]:
dub_data=dub_data.loc[dub_data["Location"].isin(["Dublin 1","Dublin 2","Dublin 3","Dublin 4",
                           "Dublin 5","Dublin 6","Dublin 6W","Dublin 7","Dublin 8",
                           "Dublin 9","Dublin 10","Dublin 11","Dublin 12",
                           "Dublin 13","Dublin 14","Dublin 15","Dublin 16"
                           ,"Dublin 17","Dublin 18","Dublin 20","Dublin 22","Dublin 24","Dun Laoghaire, Dublin"
                          ,"Blackrock, Dublin","Swords, Dublin","Malahide, Dublin","Balbriggan, Dublin","Clondalkin, Dublin 22"])]

In [ ]:
dub_data=dub_data.reset_index(drop=True)

In [ ]:
# To learn any null values and 3324 rows
dub_data.info()

In [ ]:
#To know how many unique values has each collumns
for collumnss in ["Year","Number of Bedrooms","Property Type"]:
    print(collumnss)
    print(dub_data[collumnss].value_counts())

In [ ]:
#To detect any duplicated rows
duplicate=dub_data.duplicated()
print("Count of Dublicated rows :" ,duplicate.sum())


In [ ]:
#to detecet outliers

In [ ]:
sns.boxplot(x=dub_data["VALUE"])



In [ ]:
dub_data.boxplot(column="VALUE",by="Year")

In [ ]:
#Outliers
# 1st quartile
q1 = np.percentile(dub_data["VALUE"],25)
# 3rd quartile
q3 = np.percentile(dub_data["VALUE"],75)
  # IQR
IQR = q3 - q1
# Outlier step
outlier_step = IQR * 1.5
# detect outlier and
    # store 
outliers = dub_data[(dub_data["VALUE"] < q1 - outlier_step) | (dub_data["VALUE"] > q3 + outlier_step)]
print(outliers.head(20))

In [ ]:
#Distrubution of outliers
outliers["Location"].value_counts().head(50)

In [ ]:
for n in var :
    plot_hist(n)

In [ ]:
#127 rows
outliers.info()

In [ ]:
#outliar percantege ( outliers/full data) %3 
print(127/3324)

In [ ]:
#percentage of outliers by Number of Bedrooms
x=dub_data["Number of Bedrooms"].value_counts()
y=outliers["Number of Bedrooms"].value_counts()
percentage=y/x
percentage.sort_values(ascending=False)

In [ ]:
#percentage of outliers by location
x=dub_data["Location"].value_counts()
y=outliers["Location"].value_counts()
percentage=y/x
percentage.sort_values(ascending=False)

In [ ]:
##percentage of outliers by Property Type
x=dub_data["Property Type"].value_counts()
y=outliers["Property Type"].value_counts()
percentage=y/x
percentage.sort_values(ascending=False)

In [ ]:
##percentage of outliers by Year
x=dub_data["Year"].value_counts()
y=outliers["Year"].value_counts()
percentage=y/x
percentage.sort_values(ascending=False)